# Goals

5. Build one prediction model using the ML algorithms of this course
6. Evaluate your prediction model
7. Try different ways to improve your model and show the improvements.
8. Submit code and results in Jupyter and HTML formats on canvas

### You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

##### Imports

In [40]:
import numpy as np
import pandas as pd

##### Read in Data

In [41]:
item_cat = pd.read_csv("data/item_categories.csv")
items = pd.read_csv("data/items.csv")
sales = pd.read_csv("data/sales_train.csv")
shops = pd.read_csv("data/shops.csv")

In [42]:
item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [43]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [44]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [45]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [46]:
test = pd.read_csv("data/test.csv")
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [47]:
sales["date_block_num"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

##### Checking for null values
* we can see there there are no null values in our datasets

In [48]:
item_cat.isnull().any()

item_category_name    False
item_category_id      False
dtype: bool

In [49]:
items.isnull().any()

item_name           False
item_id             False
item_category_id    False
dtype: bool

In [50]:
shops.isnull().any()

shop_name    False
shop_id      False
dtype: bool

In [51]:
sales.isnull().any()

date              False
date_block_num    False
shop_id           False
item_id           False
item_price        False
item_cnt_day      False
dtype: bool

###### Assumption
* If item_cnt_day is the number of sales that was done that day, then I will assume that anything less than zero is should be zero because sales can only be done if it happen in the positive direction not in the negative direction

##### Making the target values that are less than 0 to 0

In [52]:
np.sort(sales["item_cnt_day"].unique().astype(int))

array([ -22,  -16,   -9,   -6,   -5,   -4,   -3,   -2,   -1,    1,    2,
          3,    4,    5,    6,    7,    8,    9,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   19,   20,   21,   22,   23,   24,
         25,   26,   27,   28,   29,   30,   31,   32,   33,   34,   35,
         36,   37,   38,   39,   40,   41,   42,   43,   44,   45,   46,
         47,   48,   49,   50,   51,   52,   53,   54,   55,   56,   57,
         58,   59,   60,   61,   62,   63,   64,   65,   66,   67,   68,
         69,   70,   71,   72,   73,   74,   75,   76,   77,   78,   79,
         80,   81,   82,   83,   84,   85,   86,   87,   88,   89,   90,
         91,   92,   93,   95,   96,   97,   98,   99,  100,  101,  102,
        103,  104,  105,  106,  107,  108,  109,  110,  111,  112,  113,
        114,  115,  116,  117,  118,  121,  124,  126,  127,  128,  129,
        130,  131,  132,  133,  134,  135,  138,  139,  140,  142,  145,
        146,  147,  148,  149,  150,  151,  153,  1

In [53]:
day = np.array(sales["item_cnt_day"])
day[day < 0] = 0
sales["item_cnt_day"] = day

##### Adding another feature to the dataset which is the item category

In [16]:
item_categories = np.array(items["item_category_id"])
item_id_sales = sales["item_id"]
lst = list()
for item_identification in item_id_sales:
    lst.append(item_categories[item_identification])
sales["item_category"] = lst

##### Reformatting the date in the dataset so I can use the groupby function

In [54]:
sales['date'] = pd.to_datetime(sales["date"])
sales['month'] = sales['date'].dt.month

In [55]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,2013-02-01,0,59,22154,999.00,1.0,2
1,2013-03-01,0,25,2552,899.00,1.0,3
2,2013-05-01,0,25,2552,899.00,0.0,5
3,2013-06-01,0,25,2554,1709.05,1.0,6
4,2013-01-15,0,25,2555,1099.00,1.0,1


In [56]:
sales.drop(["date","date_block_num"],axis=1,inplace=True)

In [57]:
sales

,shop_id,item_id,item_price,item_cnt_day,month
0,59,22154,999.00,1.0,2
1,25,2552,899.00,1.0,3
2,25,2552,899.00,0.0,5
3,25,2554,1709.05,1.0,6
4,25,2555,1099.00,1.0,1
...,...,...,...,...,...
2935844,25,7409,299.00,1.0,10
2935845,25,7460,299.00,1.0,9
2935846,25,7459,349.00,1.0,10
2935847,25,7440,299.00,1.0,10


In [75]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn import metrics


x = sales.drop(["item_cnt_day"],axis=1).values
y = sales["item_cnt_day"].values
rse_lst = list()
kf = KFold()

for train_index,test_index in kf.split(x):
    x_train,x_test = x[train_index],x[test_index]
    y_train,y_test = y[train_index],y[test_index]
    pipe = make_pipeline(StandardScaler(),RandomForestRegressor())
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    rse = metrics.r2_score(y_test, y_pred)
    print('Mean Absolute Error:', mae)  
    print('Mean Squared Error:', mse)  
    print('Root Mean Squared Error:', rmse)
    print('R-squared Error:', rse)
    rse_lst.append(rse)

KeyboardInterrupt: 

##### This is what dataframe

In [61]:
mons = np.array(sales.groupby(['month',"shop_id","item_id","item_price"]).agg({'item_cnt_day':'sum'}).index.get_level_values(0))
shop_ident = np.array(sales.groupby(['month',"shop_id","item_id","item_price"]).agg({'item_cnt_day':'sum'}).index.get_level_values(1))
item_ident = np.array(sales.groupby(['month',"shop_id","item_id","item_price"]).agg({'item_cnt_day':'sum'}).index.get_level_values(2))
#date_block = np.array(sales.groupby(['month',"shop_id","item_id","item_category","date_block_num","item_price"]).agg({'item_cnt_day':'sum'}).index.get_level_values(3))
price_item = np.array(sales.groupby(['month',"shop_id","item_id","item_price"]).agg({'item_cnt_day':'sum'}).index.get_level_values(3))
cum = np.array(sales.groupby(['month',"shop_id","item_id","item_price"]).agg({'item_cnt_day':'sum'})).astype(int)

In [62]:
mons.shape

(1834856,)

In [63]:
shop_ident.shape

(1834856,)

In [64]:
item_ident.shape

(1834856,)

In [65]:
cum = cum.reshape(-1)
cum.shape

(1834856,)

In [66]:
df = pd.DataFrame({"month":mons,"shop_id":shop_ident,"item_id":item_ident,"item_price":price_item,"total_products_sold":cum})

In [67]:
df.describe().astype(int)

,month,shop_id,item_id,item_price,total_products_sold
count,1834856,1834856,1834856,1834856,1834856
mean,6,32,10477,895,1
std,3,16,6257,1822,9
min,1,0,0,-1,0
25%,3,21,4866,229,1
50%,6,31,10086,399,1
75%,9,47,15935,999,2
max,12,59,22169,307980,2453


In [76]:
df

,month,shop_id,item_id,item_price,total_products_sold
0,1,0,32,221.0,4
1,1,0,33,347.0,2
2,1,0,35,247.0,2
3,1,0,43,221.0,1
4,1,0,51,127.0,1
...,...,...,...,...,...
1834851,12,59,22091,109.0,1
1834852,12,59,22091,139.0,9
1834853,12,59,22092,109.0,8
1834854,12,59,22092,139.0,3


In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn import metrics


x = df.drop(["total_products_sold"],axis=1).values
y = df["total_products_sold"].values
rse_lst = list()
kf = KFold()

for train_index,test_index in kf.split(x):
    x_train,x_test = x[train_index],x[test_index]
    y_train,y_test = y[train_index],y[test_index]
    pipe = make_pipeline(StandardScaler(),RandomForestRegressor())
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    rse = metrics.r2_score(y_test, y_pred)
    print('Mean Absolute Error:', mae)  
    print('Mean Squared Error:', mse)  
    print('Root Mean Squared Error:', rmse)
    print('R-squared Error:', rse)
    rse_lst.append(rse)

Mean Absolute Error: 0.994687714594029
Mean Squared Error: 15.180744874813337
Root Mean Squared Error: 3.896247537671771
R-squared Error: 0.6802828303055865
Mean Absolute Error: 0.9784821688907297
Mean Squared Error: 22.215043396889676
Root Mean Squared Error: 4.713283716994944
R-squared Error: 0.6819332025463176
Mean Absolute Error: 0.8618396276545015
Mean Squared Error: 10.749650378095271
Root Mean Squared Error: 3.2786659448768596
R-squared Error: 0.8658123371437281
Mean Absolute Error: 0.9381765316605398
Mean Squared Error: 34.15199519417065
Root Mean Squared Error: 5.843970841317627
R-squared Error: 0.6961980339606056
Mean Absolute Error: 1.2212348932204453
Mean Squared Error: 41.88685967147267
Root Mean Squared Error: 6.472005846062925
R-squared Error: 0.6681499206717214


In [78]:
np.mean(rse_lst)

0.7184752649255918